In [2]:
:timing
:sccache 1
:dep poker = { path = "." }
:dep rand = "0.8.5"
:dep rand_xoshiro = "0.6.0"

Timing: true
sccache: true


Took 82031ms

In [3]:
use std::fs::File;
use std::path::Path;
use rand::SeedableRng;
use rand_xoshiro::Xoshiro256PlusPlus;
use poker::card::{self, ALL, Rank, Suit, Card, from_str, from_id, id_sum, join_id, get_joker_bits};
use poker::hand_type::{self, HandType, RANK_LOOKUP, get_hand_type, get_major_rank, get_minor_rank};
use poker::pineapple::play::{PlayRounds, BruteForceMethod};

Took 313ms

## Debug

Assume following is the test case that failed. 
```
Row0: Ks Ad As 
Row1: Ts Th 2j 5c 3d
Row2: 2s 2h 2c
```
Combine all cards into one string separated by spaces, i.e., 
```
"Ks Ad As Ts Th 2j 5c 3d 2s 2h 2c"
```
Get left cards by running the following code. 

In [4]:
join_id(ALL ^ id_sum(&from_str("Jd 6c 6d Jc 8d 7d Kd Js Jh 7h 7c").unwrap()), 54).unwrap()

"As Ah Ac Ad Ks Kh Kc Qs Qh Qc Qd Ts Th Tc Td 9s 9h 9c 9d 8s 8h 8c 7s 6s 6h 5s 5h 5c 5d 4s 4h 4c 4d 3j 3s 3h 3c 3d 2j 2s 2h 2c 2d"

Took 1045ms

In [5]:
let id0 = id_sum(&from_str("Jd 6c 6d").unwrap());
let id1 = id_sum(&from_str("Jc 8d 7d").unwrap());
let id2 = id_sum(&from_str("Kd Js Jh 7h 7c").unwrap());
let mut cards_left = from_str("As Ah Ac Ad Ks Kh Kc Qs Qh Qc Qd Ts Th Tc Td 9s 9h 9c 9d 8s 8h 8c 7s 6s 6h 5s 5h 5c 5d 4s 4h 4c 4d 3j 3s 3h 3c 3d 2j 2s 2h 2c 2d").unwrap();
cards_left.sort_by_key(|c| c.get_index());
let ids_left = id_sum(&cards_left);
assert!(ids_left & id0 == 0);
assert!(ids_left & id1 == 0);
assert!(ids_left & id2 == 0);

let mut play: PlayRounds = PlayRounds::new(id0, id1, id2);
let mut rng = Xoshiro256PlusPlus::seed_from_u64(13);
let s_it = play.score_r4_it_debug(&cards_left, 2.0, "test.csv");
let s = play.score_r4(&cards_left, 2.0, &mut BruteForceMethod::Iteration);
let delta = (s - s_it).abs();
println!("Iteration score: {}", s_it);
println!("Analytical score: {}", s);
println!("Delta: {}, Pass test: {}", delta, delta < 1E-8);

Iteration score: -6.189449801474759
Analytical score: -6.189449801474759
Delta: 0, Pass test: true


Took 5871ms

In [4]:
let id0 = id_sum(&from_str("Jd 6c 6d").unwrap());
let id1 = id_sum(&from_str("Jc 8d 7d").unwrap());
let id2 = id_sum(&from_str("Kd Js Jh 7h 7c").unwrap());
let mut cards_left = from_str("7s 6h 3c 3d").unwrap();
cards_left.sort_by_key(|c| c.get_index());
let ids_left = id_sum(&cards_left);
assert!(ids_left & id0 == 0);
assert!(ids_left & id1 == 0);
assert!(ids_left & id2 == 0);

let mut play: PlayRounds = PlayRounds::new(id0, id1, id2);
let mut rng = Xoshiro256PlusPlus::seed_from_u64(13);
let s_it = play.score_r4_it_debug(&cards_left, 2.0, "test.csv");
let s = play.score_r4(&cards_left, 2.0, &mut BruteForceMethod::Iteration);
let delta = (s - s_it).abs();
println!("Iteration score: {}", s_it);
println!("Analytical score: {}", s);
println!("Delta: {}, Pass test: {}", delta, delta < 1E-8);

Took 5371ms

Iteration score: -2.25
Analytical score: -2.25
Delta: 0, Pass test: true
